In [7]:
import tarfile
from pathlib import Path
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import shutil
import multiprocessing

# === Use container disk (/tmp) for faster IO ===
split_dir = Path('.')  # You moved .tar.gz files here
base_dir = split_dir / 'splits_simple_fused' / 'splits_simple_fused_2025-06-06_01-11-05'
base_dir.mkdir(parents=True, exist_ok=True)

# === List your .tar.gz archives ===
split_archives = [
    'splits_simple_fused_2025-06-06_01-11-05_images_trains.tar.gz',
    'splits_simple_fused_2025-06-06_01-11-05_lables_trains.tar.gz',
    'splits_simple_fused_2025-06-06_01-11-05_val_test.tar.gz'
]
tar_files = [split_dir / fname for fname in split_archives]

# === Extraction function ===
def extract_tar(tar_path):
    print(f"🔓 Starting: {tar_path.name}")
    with tarfile.open(tar_path, "r:gz") as tar:
        members = tar.getmembers()
        total = len(members)
        print(f"📁 {total} files in {tar_path.name}")
        for i, member in enumerate(members, 1):
            tar.extract(member, path=base_dir, filter='data')
            if i % 10000 == 0:
                print(f"📈 {tar_path.name}: {i:,}/{total:,} ({(i/total)*100:.1f}%)")
    print(f"✅ Done: {tar_path.name}")
    return tar_path.name

# === Parallel Extraction ===
print(f"🔍 Extracting {len(tar_files)} archives...")
workers = min(len(tar_files), multiprocessing.cpu_count())
with ThreadPoolExecutor(max_workers=workers) as executor:
    futures = [executor.submit(extract_tar, f) for f in tar_files]
    for i, f in enumerate(futures, 1):
        f.result()
        print(f"📦 Completed {i}/{len(futures)}")

# === Create YOLO folder structure ===
image_train = base_dir / 'images' / 'train'
label_train = base_dir / 'labels' / 'train'
image_val = base_dir / 'images' / 'val'
label_val = base_dir / 'labels' / 'val'
image_test = base_dir / 'images' / 'test'
label_test = base_dir / 'labels' / 'test'

for path in [image_train, label_train, image_val, label_val, image_test, label_test]:
    path.mkdir(parents=True, exist_ok=True)

# === Move Files ===
def move_all(src_glob, dst_folder, description):
    count = 0
    for file in base_dir.glob(src_glob):
        shutil.move(str(file), dst_folder / file.name)
        count += 1
    print(f"📦 Moved {count} {description} to {dst_folder}")

move_all("**/images/train/*.png", image_train, "train images")
move_all("**/labels/train/*.txt", label_train, "train labels")
move_all("**/images/val/*.png", image_val, "val images")
move_all("**/labels/val/*.txt", label_val, "val labels")
move_all("**/images/test/*.png", image_test, "test images")
move_all("**/labels/test/*.txt", label_test, "test labels")

print(f"\n✅ Done. Folder ready at: {base_dir}")

🔍 Extracting 3 archives...
🔓 Starting: splits_simple_fused_2025-06-06_01-11-05_images_trains.tar.gz
🔓 Starting: splits_simple_fused_2025-06-06_01-11-05_lables_trains.tar.gz
🔓 Starting: splits_simple_fused_2025-06-06_01-11-05_val_test.tar.gz
📁 221705 files in splits_simple_fused_2025-06-06_01-11-05_val_test.tar.gz
📈 splits_simple_fused_2025-06-06_01-11-05_val_test.tar.gz: 10,000/221,705 (4.5%)
📁 629594 files in splits_simple_fused_2025-06-06_01-11-05_lables_trains.tar.gz
📈 splits_simple_fused_2025-06-06_01-11-05_val_test.tar.gz: 20,000/221,705 (9.0%)
📈 splits_simple_fused_2025-06-06_01-11-05_lables_trains.tar.gz: 10,000/629,594 (1.6%)
📈 splits_simple_fused_2025-06-06_01-11-05_val_test.tar.gz: 30,000/221,705 (13.5%)
📈 splits_simple_fused_2025-06-06_01-11-05_lables_trains.tar.gz: 20,000/629,594 (3.2%)
📁 629594 files in splits_simple_fused_2025-06-06_01-11-05_images_trains.tar.gz
📈 splits_simple_fused_2025-06-06_01-11-05_val_test.tar.gz: 40,000/221,705 (18.0%)
📈 splits_simple_fused_2025-06

In [1]:
from pathlib import Path
dataset_root = Path('./splits_simple_fused/splits_simple_fused_2025-06-06_01-11-05')

splits = ['train', 'val', 'test']
for split in splits:
    img_dir = dataset_root / 'images' / split
    lbl_dir = dataset_root / 'labels' / split

    img_files = list(img_dir.glob('*.png'))
    lbl_files = list(lbl_dir.glob('*.txt'))

    print(f"\n📂 {split.upper()}")
    print(f"🖼️  Images: {len(img_files)}")
    print(f"🏷️  Labels: {len(lbl_files)}")

    img_stems = {f.stem for f in img_files}
    lbl_stems = {f.stem for f in lbl_files}
    missing_labels = img_stems - lbl_stems
    missing_images = lbl_stems - img_stems

    if missing_labels:
        print(f"⚠️ Missing labels for: {list(missing_labels)[:5]}")
    if missing_images:
        print(f"⚠️ Missing images for: {list(missing_images)[:5]}")
    if not missing_labels and not missing_images:
        print("✅ All image-label pairs match.")


📂 TRAIN
🖼️  Images: 629591
🏷️  Labels: 629591
✅ All image-label pairs match.

📂 VAL
🖼️  Images: 72528
🏷️  Labels: 72528
✅ All image-label pairs match.

📂 TEST
🖼️  Images: 38321
🏷️  Labels: 38321
✅ All image-label pairs match.


In [2]:
import yaml

class_names = ["chair", "box", "desk", "door_frame"]

data_yaml = {
    'path': str(dataset_root),
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',
    'nc': len(class_names),
    'names': class_names
}

yaml_file_path = dataset_root / 'data.yaml'
with open(yaml_file_path, 'w') as f:
    yaml.dump(data_yaml, f)

print(f"📄 data.yaml saved at: {yaml_file_path}")

📄 data.yaml saved at: splits_simple_fused/splits_simple_fused_2025-06-06_01-11-05/data.yaml


In [3]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

results = model.train(
    data=str(yaml_file_path),
    epochs=120,
    batch=1024,         # Safe value for 40 GB VRAM
    imgsz=384,
    workers=10,          # Balanced for 30+ CPU cores
    rect=True,          # All images are same size (64x384)
    cache='ram',        # Leverage 200 GB RAM for faster training
    seed=42,
    save=True,
    save_period=10,

    # ✅ Safe augmentations
    flipud=0.2,
    hsv_h=0.0,
    hsv_s=0.0,
    hsv_v=0.0,

    # ❌ Disable risky augmentations
    mosaic=0.0,         # Disable mosaic
    mixup=0.0,          # Disable mixup
    copy_paste=0.0,     # Disable copy-paste
    cutmix=0.0,         # Disable cutmix
    auto_augment=None,  # Disable auto augment policies
    augment=False,      # Disable general heavy augmentation
    erasing=0.0,        # Disable random erasing

    verbose=True
)

print("✅ Training complete!")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/home/soheil/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.152 🚀 Python-3.10.12 torch-2.7.1+cu126 CUDA:0 (NVIDIA H100 80GB HBM3, 81008MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=None, batch=1024, bgr=0.0, box=7.5, cache=ram, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=splits_simple_fused/splits_simple_fused_2025-06-06_01-11-05/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=120, erasing=0.0, exist_ok=False, fliplr=0.5, flipud=0.2, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, i

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 755k/755k [00:00<00:00, 138MB/s]

Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics


[W610 11:29:18.105124366 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W610 11:29:18.115956094 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W610 11:29:18.118270869 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W610 11:29:18.121304390 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W610 11:29:18.122837323 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W610 11:29:18.123086057 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W610 11:29:18.124662972 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W610 11:29:18.125020844 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W610 11:29:18.125508397 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W610 11:29:18.126045196 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W610 11:

Model summary: 129 layers, 3,011,628 parameters, 3,011,612 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights


...

                   all      72528     276961     0.0153     0.0824     0.0122    0.00473

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    113/120      10.5G      0.493     0.2869     0.8602       3132        384: 100%|██████████| 615/615 [02:40<00:00,  3.82it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 36/36 [02:05<00:00,  3.48s/


                   all      72528     276961    0.00903      0.322      0.012     0.0047
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 13, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

113 epochs completed in 9.116 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.152 🚀 Python-3.10.12 torch-2.7.1+cu126 CUDA:0 (NVIDIA H100 80GB HBM3, 81008MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 36/36 [02:43<00:00,  4.53s/


                   all      72528     276961       0.95      0.949      0.984       0.78
                 chair      71830      71830      0.986      0.922      0.983       0.75
                   box      71999      71999      0.974      0.995      0.994      0.805
                  desk      72509      72509      0.921      0.956      0.981      0.805
            door_frame      60623      60623      0.919      0.924      0.978       0.76
Speed: 0.0ms preprocess, 0.0ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train
✅ Training complete!


In [7]:
from ultralytics import YOLO
import torch
import json
import shutil
import os
from onnxruntime.quantization import quantize_dynamic, QuantType

# --- Paths ---
weights_dir = 'runs/detect/train/weights'
best_model_path = f'{weights_dir}/best.pt'
data_yaml_path = './splits_simple_fused/splits_simple_fused_2025-06-06_01-11-05/data.yaml'
webots_dir = './webots_model_assets_simpel_fused_fulltrain_120'
os.makedirs(webots_dir, exist_ok=True)

# --- Load Trained YOLOv8 Model ---
model = YOLO(best_model_path)

# --- Export TorchScript (FP32) ---
print("🔁 Exporting TorchScript...")
model.export(format='torchscript', imgsz=[64, 384])
torchscript_path = f'{weights_dir}/best.torchscript'

# --- Dynamic Quantization for TorchScript ---
print("⚙️ Quantizing TorchScript model...")
model_fp32 = torch.jit.load(torchscript_path)
model_int8 = torch.quantization.quantize_dynamic(model_fp32, {torch.nn.Linear}, dtype=torch.qint8)
torchscript_quant_path = f'{weights_dir}/best_quant_dynamic.torchscript'
torch.jit.save(model_int8, torchscript_quant_path)

# --- Export ONNX (FP32) ---
print("🔁 Exporting ONNX...")
model.export(format='onnx', imgsz=[64, 384], dynamic=True)
onnx_path = f'{weights_dir}/best.onnx'

# --- Dynamic Quantization for ONNX ---
print("⚙️ Quantizing ONNX model...")
onnx_quant_path = f'{weights_dir}/best_quant_dynamic.onnx'
quantize_dynamic(onnx_path, onnx_quant_path, weight_type=QuantType.QInt8)

# --- Export TensorRT (FP16) ---
print("🚀 Exporting TensorRT (FP16)...")
model.export(format='engine', imgsz=[64, 384])
tensorrt_path = f'{weights_dir}/best.engine'

# --- Evaluate model ---
print("🔍 Evaluating on test set...")
metrics = model.val(data=data_yaml_path, split='test')

# --- Save metrics ---
metrics_path = 'test_metrics.json'
with open(metrics_path, 'w') as f:
    json.dump(metrics.results_dict, f, indent=2)
print("✅ Metrics saved.")

# --- Copy all files to Webots directory ---
shutil.copy(best_model_path, f'{webots_dir}/yolov8n_lidar.pt')
shutil.copy(torchscript_path, f'{webots_dir}/yolov8n_lidar.torchscript')
shutil.copy(torchscript_quant_path, f'{webots_dir}/yolov8n_lidar_quant_dynamic.torchscript')
shutil.copy(onnx_path, f'{webots_dir}/yolov8n_lidar.onnx')
shutil.copy(onnx_quant_path, f'{webots_dir}/yolov8n_lidar_quant_dynamic.onnx')
shutil.copy(tensorrt_path, f'{webots_dir}/yolov8n_lidar_fp16.engine')
shutil.copy(metrics_path, f'{webots_dir}/test_metrics.json')

print(f"📦 All models and metrics saved to: {webots_dir}")

🔁 Exporting TorchScript...
Ultralytics 8.3.152 🚀 Python-3.10.12 torch-2.7.1+cu126 CPU (Intel Xeon Platinum 8468)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs/detect/train/weights/best.pt' with input shape (1, 3, 64, 384) BCHW and output shape(s) (1, 8, 504) (5.9 MB)

TorchScript: starting export with torch 2.7.1+cu126...


[W610 20:51:20.992718532 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W610 20:51:20.993261917 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W610 20:51:20.993422053 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W610 20:51:20.993593112 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W610 20:51:20.993719941 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W610 20:51:20.993861771 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W610 20:51:20.994007700 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W610 20:51:20.994139457 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W610 20:51:20.994276295 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W610 20:51:20.994390880 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W610 20:5

TorchScript: export success ✅ 0.7s, saved as 'runs/detect/train/weights/best.torchscript' (11.8 MB)

Export complete (0.8s)
Results saved to /home/soheil/runs/detect/train/weights
Predict:         yolo predict task=detect model=runs/detect/train/weights/best.torchscript imgsz=64,384  
Validate:        yolo val task=detect model=runs/detect/train/weights/best.torchscript imgsz=64,384 data=splits_simple_fused/splits_simple_fused_2025-06-06_01-11-05/data.yaml  WARNING ⚠️ non-PyTorch val requires square images, 'imgsz=[64, 384]' will not work. Use export 'imgsz=384' if val is required.
Visualize:       https://netron.app


[W610 20:51:21.690870112 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W610 20:51:21.690960879 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W610 20:51:21.691033740 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W610 20:51:21.691094176 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W610 20:51:21.691487333 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W610 20:51:21.691539000 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W610 20:51:21.691595984 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W610 20:51:21.691661020 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W610 20:51:21.691713967 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W610 20:51:21.691930985 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W610 20:5

⚙️ Quantizing TorchScript model...
🔁 Exporting ONNX...
Ultralytics 8.3.152 🚀 Python-3.10.12 torch-2.7.1+cu126 CPU (Intel Xeon Platinum 8468)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs/detect/train/weights/best.pt' with input shape (1, 3, 64, 384) BCHW and output shape(s) (1, 8, 504) (5.9 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<1.18.0', 'onnxslim>=0.1.56', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
WARNING ⚠️ requirements: ❌ [Errno 13] Permission denied: 'uv'

ONNX: starting export with onnx 1.18.0 opset 19...


...

val: Fast image access ✅ (ping: 0.3±0.2 ms, read: 5.4±2.1 MB/s, size: 0.8 KB)


val: Scanning /home/soheil/splits_simple_fused/splits_simple_fused_2025-06-06_01-11-05/labels/test... 38321 images, 0 backgrounds, 0 c


val: New cache created: /home/soheil/splits_simple_fused/splits_simple_fused_2025-06-06_01-11-05/labels/test.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2396/2396 [01:29<00:00, 26.


                   all      38321     146223      0.949      0.947      0.984      0.778
                 chair      37699      37699      0.986      0.913      0.981      0.746
                   box      37917      37917      0.957      0.996      0.993       0.78
                  desk      38234      38234      0.922      0.954      0.982      0.822
            door_frame      32373      32373       0.93      0.926       0.98      0.763
Speed: 0.0ms preprocess, 0.2ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/val
✅ Metrics saved.
📦 All models and metrics saved to: ./webots_model_assets_simpel_fused_fulltrain_120


In [8]:
import shutil

# Zip the entire runs directory
shutil.make_archive('./runs_yolo_120', 'zip', './runs')
shutil.make_archive('./webots_model_assets_simpel_fused_fulltrain_120', 'zip', './webots_model_assets_simpel_fused_fulltrain_120')

'/home/soheil/webots_model_assets_simpel_fused_fulltrain_120.zip'